In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import re
import statsmodels.formula.api as smf
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [3]:
import session_info
session_info.show()

In [2]:
wsize = "5Mb"
dat = pd.read_csv(f"../data/features2windows/{wsize}.merged.tab.gz", sep="\t", na_values=".")
sig = pd.read_csv(f"../data/features2windows/{wsize}.signatures.tab.gz", sep="\t", na_values=".")

dat["id"] = dat.chrom + dat.start.astype("str")
sig["id"] = sig.chrom + sig.start.astype("str")

#dat = pd.merge(dat, sig, on="id")

In [3]:
def fit_exp_decay(x, y0):
    y = np.log(np.array(y0) + 1)
    slope, intercept, _, _, _ = linregress(x, y)
    return intercept, slope

def infer_repair_rate(row: pd.Series, columns: list) -> float:

    times = []
    dmgs = []

    for col in columns:
        match = re.search(r'_(\d+)_', col) # Extracts time from column name
        if match:
            t = int(match.group(1))
            # Handle cases where the same time point has multiple replicates
            times.append(t)
            dmgs.append(row[col])
        else:
            print(f"Warning: Could not parse time from column: {col}")

    if not times or not dmgs or len(times) < 2:
        return np.nan # Not enough data to fit

    # Sort by time to ensure correct fitting
    sorted_indices = np.argsort(times)
    t_data = np.array(times)[sorted_indices]
    y_data = np.array(dmgs)[sorted_indices]

    _,slope = fit_exp_decay(t_data, y_data)

    return slope

In [ ]:
col_remap = {"ESC_Rep1.bed.gz-ESC_1": "rep_timing_1", 
             "ESC_Rep2.bed.gz-ESC_2": "rep_timing_2", 
             "ESC_Rep1.bed.gz-ESC_3": "rep_timing_3", 
             "GSE113957_fpkm_xL_sorted.bed-GSE113957_fpkm_xL_sorted": "total_expression_fibroblasts", 
             "hg19_refGene_transcripts_merged_overlap_count":"gene_perc", 
             "20141020_strict_mask_whole_genome_overlap_count": "complex_regions_perc", 
             "Jiang2023-bpde_Cell_0_25uM_1": "Jiang2023-bpde_Cell_0.25uM_1", 
             "Jiang2023-bpde_Cell_0_25uM_2": "Jiang2023-bpde_Cell_0.25uM_2", 
             "Jiang2023-bpde_Cell_0_5uM_1": "Jiang2023-bpde_Cell_0.5uM_1", 
             "Jiang2023-bpde_Cell_0_5uM_2": "Jiang2023-bpde_Cell_0.5uM_2"}
dat = dat.rename(columns = col_remap)
dat = dat.rename(columns = {c:c.replace("Yoshida2020-", "") 
                            for c in dat.columns if "Yoshida2020" in c})

# Get rid of windows with (mostly) just gaps & short lenghts
dat = dat[(dat.N_content<100e3) & 
          (dat.total_len>500e3)].reset_index(drop=True)

# Ratio of SBS5
dat["sbs5_ratio"] = dat["ever_smoker_sbs5"] / dat["non_smoker_sbs5"]
dat["sbs5_sbs4_ratio"] = np.log2(dat.ever_smoker_sbs5 / dat.ever_smoker_sbs4)

# Norm by median
for smoke in ["ever_smoker", "non_smoker", "smoker"]:
    for sbs in ["sbs5", "sbs4", "sbs40"]:
        dat[f"{smoke}_{sbs}_norm"] = dat[f"{smoke}_{sbs}"] / dat[f"{smoke}_{sbs}"].median()

# Expression per base-pair
denominator = dat.gene_perc * dat.total_len
dat["expression_rate"] = np.where(
    denominator != 0,
    dat.total_expression_fibroblasts / denominator,
    np.nan  # or 0, depending on what makes sense for your case
)

# NER early readouts
hu_early_ner = ["Hu2017-cpd_0_1", "Hu2017-cpd_0_2", "Hu2017-cpd_60_1", "Hu2017-cpd_60_2"]
dat["cpd_ner_hu"] = dat[hu_early_ner].sum(axis=1)
li_early_ner = ["Li2017-cpd_240_1", "Li2017-cpd_240_2"]
dat["cpd_ner_li"] = dat[li_early_ner].sum(axis=1)
li_bpde_ner = ["Li2017-bpde_60_1", "Li2017-bpde_60_2"]
dat["bpde_ner_li"] = dat[li_bpde_ner].sum(axis=1)

# Damage
dat["bpde_naked_2uM"] = dat["Jiang2023-bpde_nDNA_2uM_1"] + dat["Jiang2023-bpde_nDNA_2uM_2"]
dat["bpde_cell_2uM"] = dat["Jiang2023-bpde_Cell_2uM_1"] + dat["Jiang2023-bpde_Cell_2uM_2"]

# Repair
dmg_cols = [
    'Hu2017-cpd_0_1',
    'Hu2017-cpd_0_2',
    'Hu2017-cpd_1440_1',
    'Hu2017-cpd_1440_2',
    'Hu2017-cpd_2160_1',
    'Hu2017-cpd_2160_2',
    'Hu2017-cpd_2880_1',
    'Hu2017-cpd_2880_2',
    'Hu2017-cpd_480_1',
    'Hu2017-cpd_480_2',
    'Hu2017-cpd_60_1',
    'Hu2017-cpd_60_2']

dat["cpd_repair"] = dat.apply(infer_repair_rate, axis=1, columns=dmg_cols) * -1